# Python setup #

The work developed using DeepTCR has been carried out in a virtual Python 3.7.17 environment, so that the DeepTCR dependencies are correctly installed, and therefore the library itself. The execution of DeepTCR has been done using Jupyter notebooks in VScode. In addition to DeepTCR it is necessary to import the sys, pandas, numpy and pickle libraries.


In [ ]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../../')
from DeepTCR.DeepTCR import DeepTCR_WF
import pickle



# Workflow #

The pipeline consists of (i), creation or reloading of the work area via Get_Data(), (ii) creation of the repertoire classification model via Monte_Carlo_CrossVal(), and (iii) representation of the AUC-ROC curve. In case of positive results in the classification model, the pipeline has continued with (iv) identification of motifs by Motif_Identification() and obtaining the Residue Sensitivity Logos for each cluster (v). This workflow has been applied in all supervised analysis scripts. The procedure has been developed given the scripts deposited at https://github.com/sidhomj/DeepTCR_COVID19, whose results are published at https://doi.org/10.1038/s41598-021-93608-8



### 1. Getting data for the comparison Cluster 1 vs Cluster 3 ###

In [6]:
DTCR_CT13 = DeepTCR_WF('CT1_CT3')
classes = ['ct1', 'ct3'] # Definition of classes of samples we compare
DTCR_CT13.Get_Data('ct1_ct3',
              Load_Prev_Data=True,
              aa_column_beta=0,
              v_beta_column=2,
              j_beta_column=3,
              count_column=1,
              data_cut=1000, # Selecting to 1000 expanded clonotypes
              type_of_data_cut='Num_Seq',
              aggregate_by_aa=True, # Preventing redundancy bias of repeated clonotypes by aminoacid sequence
              classes=classes)

Data Loaded


### 2. Creating repertorie classification model ###

We used a default 75%/25% tran/test split, with 100-fold Monte Carlo cross-validation and subsampling of 100 sequences for regularisation and speed improvement. We also enabled class weighting to handle class imbalance and multisample dropout with default parameters for better training and generalisation, as described in https://arxiv.org/abs/1905.09788. 
All information about the Monte_Carlo_CrossVal() function arguments, along with the rest of DeepTCR's functions, can be found at https://sidhomj.github.io/DeepTCR/api/. 

In [ ]:
folds = 100 # 100 fold Monte-Carlo cross validation
epochs_min = 25
size_of_net = 'small'
num_concepts=64 # Tested as the best option for a better model's perfomance for our data
hinge_loss_t = 0.1
train_loss_min=0.1
seeds = np.array(range(folds))
graph_seed = 0

DTCR_CT13.Monte_Carlo_CrossVal(folds=folds,epochs_min=epochs_min,size_of_net=size_of_net,num_concepts=num_concepts,
                          train_loss_min=train_loss_min,combine_train_valid=True,
                          hinge_loss_t=hinge_loss_t,
                          multisample_dropout=True, # Enabling multi-sample dropout rate
                          weight_by_class =True, # Enabling by class weigthing
                          seeds=seeds,graph_seed=graph_seed,
                            subsample=100 ) # Subsampling 100 sequences each fold during training

### 3. AUC curve plotting of the model results ###
The AUC-ROC curve corresponds to the results of the test partition performance

In [ ]:
DTCR_CT13.AUC_Curve(filename='AUC_ct1ct3.tif', title='AUC-ROC curves from Cluster 1 and cluster 3 Samples')

### 4. Motif identification ###
We used the default p-value threshold of 0.05 for enriched motifs by cluster (Mann-Whitney U Test) at the sample level. 

In [ ]:
DTCR_CT13.Motif_Identification('ct1', by_samples=True) # Enablig by sample motif identification instead of at sequence level
DTCR_CT13.Motif_Identification('ct3', by_samples=True)

### 5. Residue Sensitivity Logos ###

Since the model performed well on the test partition, we delved deeper into the characterisation of the CDR3b signatures by RSLs. First, we stored the AUC of the prediction that the model made for each CDR3b for each cluster. We then selected the top 25 sequences with the highest AUC in each cluster (two sets of 25) and created the RSL using the Residue_Sensitivity_Logo() function. 

In [ ]:
# Data frame of each sequence's AUC for each clusters during model performance using the test partition

with open('ct13_model_preds.pkl','wb') as f:
    pickle.dump(DTCR_CT13.DFs_pred,f,protocol=4)

with open('ct13_model_seq_preds.pkl', 'wb') as f:
    pickle.dump((DTCR_CT13.predicted,DTCR_CT13.beta_sequences,DTCR_CT13.lb), f, protocol=4) 

with open('ct13_model_seq_preds.pkl', 'rb') as f:
    predicted, beta_sequences,lb = pickle.load(f)

df = pd.DataFrame()
df['beta_sequences'] = beta_sequences
df[lb.classes_[0]] = predicted[:,0]
df[lb.classes_[1]] = predicted[:,1]

#### 5.1 RSL for cluster 1 #####

The data frame created above is sorted in such a way that the sequences with the highest AUC for cluster 1 appear first (descending order). The corresponding pandas functions are used for this purpose.

In [ ]:
sel = 0
cl = lb.classes_[sel]
df.sort_values(by=cl,inplace=True,ascending=False)

In [ ]:
fig13_ct1,ax = DTCR_CT13.Residue_Sensitivity_Logo(beta_sequences=np.array(df['beta_sequences'])[0:25], # top 25 sequences with the highest AUC for cluster 1,
                              class_sel=cl,figsize=(5,10),background_color='white',Load_Prev_Data=False,
                              min_size=0.25,  
                              low_color='blue', medium_color='yellow', high_color='red' # color settings
                              )


In [ ]:
# Saving the RSL
fig13_ct1.savefig('rsl_ct13_ct1.png',
            dpi=1200,facecolor='black')

##### 5.2 RSL for cluster 3 ####

In [ ]:
sel = 0
cl = lb.classes_[sel]
df.sort_values(by=cl,inplace=True,ascending=False)

In [ ]:
fig13_ct3,ax = DTCR_CT13.Residue_Sensitivity_Logo(beta_sequences=np.array(df['beta_sequences'])[0:25], # top 25 sequences with the highest AUC for cluster 3,
                              class_sel=cl,figsize=(5,10),background_color='white',Load_Prev_Data=False,
                              min_size=0.25,  low_color='blue', medium_color='yellow', high_color='red')
fig13_ct3.savefig('rsl_ct13_ct1.png',
            dpi=1200,facecolor='black')